In [1]:
import pandas as pd
import numpy as np
import ast
import warnings
from tqdm import tqdm
import re
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/god/data/'
dataset = pd.read_csv(path + 'dataset_for_vector.csv')

In [4]:
# sentence와 extractive 컬럼의 string을 리스트로 변환
import ast

dataset['sentence'] = dataset['sentence'].apply(ast.literal_eval)
dataset['extractive'] = dataset['extractive'].apply(ast.literal_eval)

In [5]:
# extractive에 해당하는 문장 뽑는 함수 구현
def extract_sentence(row):
    return [row['sentence'][index] for index in row['extractive']]

dataset['ex_sentence'] = dataset.apply(extract_sentence, axis=1)

In [6]:
dataset.head()

,title,sentence,extractive,ex_sentence
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...
1,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",[여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛...
2,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...","[3, 5, 7]",[임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표...
3,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","[2, 3, 4]",[광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 105...
4,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma...","[3, 7, 4]",[전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급아쿠아리우스호가 ...


# Vector

In [7]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00


In [8]:
from transformers import *
from transformers import BertTokenizer, TFBertModel, BertConfig

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

loading weights file tf_model.h5 from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tf_model.h5
Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for pre

In [19]:
def extract_sentence_vector(sentence):
    tokens = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='tf')
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    outputs = model({'input_ids': input_ids, 'attention_mask': attention_mask})
    sentence_vector = tf.reduce_mean(outputs.last_hidden_state, axis=1)
    return sentence_vector

In [43]:
df = dataset[:1]

In [44]:
df

,title,sentence,extractive,ex_sentence
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...


In [53]:
df['sentence_vectors'] = df['ex_sentence'].apply(lambda x: [extract_sentence_vector(sentence) for sentence in x])

<ipython-input-53-b6813e994f90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentence_vectors'] = df['ex_sentence'].apply(lambda x: [extract_sentence_vector(sentence) for sentence in x])


In [54]:
df['sentence_vectors'].shape

(1,)

In [55]:
df.head()

,title,sentence,extractive,ex_sentence,sentence_vectors
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...,"[((tf.Tensor(-0.16752474, shape=(), dtype=floa..."


In [56]:
df['sentence_vectors'][0]

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-1.67524740e-01, -4.47816908e-01,  2.71614313e-01,
          2.04779297e-01, -9.78576243e-02,  5.14094830e-01,
          8.42052475e-02,  3.53514284e-01, -7.61882305e-01,
          3.44829597e-02, -2.88357019e-01,  5.43647945e-01,
          5.80853164e-01,  1.16637044e-01,  1.57251790e-01,
         -4.87465113e-01,  6.95749104e-01, -1.59800172e-01,
          1.04516394e-01,  1.05732620e-01,  1.00139603e-01,
          1.79335028e-01, -1.59515291e-01,  3.75334024e-01,
          4.86228913e-02,  4.19679403e-01,  2.42427871e-01,
          7.28132576e-02, -3.65734063e-02, -1.98649809e-01,
         -4.24862914e-02,  1.21868394e-01, -4.83313769e-01,
          4.26794201e-01, -3.81867498e-01,  3.14934105e-01,
         -3.01242471e-01, -8.93326551e-02, -2.93405563e-01,
          2.38873512e-01, -5.41901767e-01,  6.68653965e-01,
         -2.06035510e-01, -2.91130662e-01, -3.33035678e-01,
          5.12340128e-01,  3.12530607e-01, -5.392

In [57]:
df2 = dataset[:1]

In [58]:
def extract_sentence_vector2(sentences):
    sentence_vectors = []
    for sentence in sentences:
        tokens = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='tf')
        input_ids = tokens['input_ids']
        attention_mask = tokens['attention_mask']
        outputs = model({'input_ids': input_ids, 'attention_mask': attention_mask})
        sentence_vector = tf.reduce_mean(outputs.last_hidden_state, axis=1)
        sentence_vectors.append(sentence_vector)
    return sentence_vectors

In [62]:
df2['sentence_vectors'] = df2['ex_sentence'].apply(extract_sentence_vector2)

<ipython-input-62-263922e712eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['sentence_vectors'] = df2['ex_sentence'].apply(extract_sentence_vector2)


In [69]:
df2['sentence_vectors'].shape

(1,)

In [66]:
df2.head()

,title,sentence,extractive,ex_sentence,sentence_vectors
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...,"[((tf.Tensor(-0.16752474, shape=(), dtype=floa..."


In [67]:
df2['sentence_vectors'][0][0]

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.67524740e-01, -4.47816908e-01,  2.71614313e-01,
         2.04779297e-01, -9.78576243e-02,  5.14094830e-01,
         8.42052475e-02,  3.53514284e-01, -7.61882305e-01,
         3.44829597e-02, -2.88357019e-01,  5.43647945e-01,
         5.80853164e-01,  1.16637044e-01,  1.57251790e-01,
        -4.87465113e-01,  6.95749104e-01, -1.59800172e-01,
         1.04516394e-01,  1.05732620e-01,  1.00139603e-01,
         1.79335028e-01, -1.59515291e-01,  3.75334024e-01,
         4.86228913e-02,  4.19679403e-01,  2.42427871e-01,
         7.28132576e-02, -3.65734063e-02, -1.98649809e-01,
        -4.24862914e-02,  1.21868394e-01, -4.83313769e-01,
         4.26794201e-01, -3.81867498e-01,  3.14934105e-01,
        -3.01242471e-01, -8.93326551e-02, -2.93405563e-01,
         2.38873512e-01, -5.41901767e-01,  6.68653965e-01,
        -2.06035510e-01, -2.91130662e-01, -3.33035678e-01,
         5.12340128e-01,  3.12530607e-01, -5.39263606e-01,
      

In [ ]:
dataset['sentence_vectors'] = dataset['ex_sentence'].apply(lambda x: [extract_sentence_vector(sentence) for sentence in x])

# vector_dataset.csv로 내보내기
dataset.to_csv('/content/drive/MyDrive/god/vector_dataset.csv',index=False)
print('데이터 저장 완료!')

In [ ]:
# 결과 확인
print(dataset['sentence_vectors'])